In [ ]:
import os
import time
import numpy as np
import wurlitzer

import trtlab
import infer_test_utils as utils

# this allows us to capture stdout and stderr from the backend c++ infer-runtime
display_output = wurlitzer.sys_pipes

In [ ]:
inputs = utils.load_inputs("/work/models/onnx/mnist-v1.3/test_data_set_0")
expected = utils.load_outputs("/work/models/onnx/mnist-v1.3/test_data_set_0")

# Remote Inference Setup

In [ ]:
with display_output():
    manager = trtlab.RemoteInferenceManager(hostname="localhost:50052")

In [ ]:
with display_output():
    models = manager.get_models()

In [ ]:
with display_output():
    print(models)

# Remote Inference Properties

In [ ]:
mnist = manager.infer_runner("mnist")

In [ ]:
mnist.input_bindings()

In [ ]:
mnist.output_bindings()

# Remote Inference Compute

In [ ]:
futures = [mnist.infer(Input3=input) for input in inputs]

In [ ]:
# free to do other work while inference is being computed

In [ ]:
results = [f.get() for f in futures]

In [ ]:
for r, e in zip(results, expected):
    for key, val in r.items():
        r = val.reshape((1,10))
        np.testing.assert_almost_equal(r, e, decimal=3)
        print("Test Passed")
        print("Output Binding Name: {}; shape: {}".format(key, val.shape))
        print("Result: {}".format(np.argmax(utils.softmax(r))))
        
utils.mnist_image(inputs[0]).show()
expected[0]